In [1]:
import numpy as np 
import pandas as pd 
import module

In [2]:
data_gpt_eval = pd.read_csv('hidden_data/CT-Repo-With-Examples-Corrected-allgpteval.csv')
#data_bert_eval = pd.read_csv('hidden_data/CT-Pub-With-Examples-Corrected-allbert07eval.csv')
data_gen = pd.read_csv('hidden_data/CT-Repo-With-Examples-Corrected-allgen.csv')

In [3]:
data_gen.head(3)

,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,TrialGroup,API_BaselineMeasures,API_BaselineMeasures_Corrected,gpt4o_zs_gen,gpt4o_ts_gen,llama3_70b_it_zs_gen,llama3_70b_it_ts_gen
0,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Inclusion Criteria:\n\n* Diagnosed with type 2...,The purpose of this study is to prevent major ...,"Atherosclerosis, Cardiovascular Diseases, Hype...","Anti-hyperglycemic Agents, Anti-hypertensive A...",First Occurrence of a Major Cardiovascular Eve...,hypertension,"Age, Continuous, Gender, Ethnicity (NIH/OMB), ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","`Age`, `Sex`, `Race/Ethnicity`, `Body Mass Ind...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","`Age`, `Sex`, `Race`, `Ethnicity`, `Body Mass ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O..."
1,NCT00003901,Prognostic Study of Metastases in Patients Wit...,Inclusion Criteria:\n\n1. Patient must be ≥ 18...,RATIONALE: Prognostic testing for early signs ...,"Lung Cancer,","immunohistochemistry staining method, biopsy, ...",Overall Survival in Lymph Nodes Examined Patie...,cancer,"Age, Continuous, Gender, Race/Ethnicity, Custo...","`Age, Continuous`, `Gender`, `Race/Ethnicity, ...","`Age`, `Gender`, `ECOG/Zubrod status`, `Clinic...","`Age, Continuous`, `Sex: Female, Male`, `Race/...","`age`, `sex`, `ECOG/Zubrod status`, `histologi...","`Age, Continuous`, `Sex: Female, Male`, `Race/..."
2,NCT00005879,LY353381 in Preventing Breast Cancer in Women ...,DISEASE CHARACTERISTICS:\n\n* Current random f...,RATIONALE: Chemoprevention therapy is the use ...,"Breast Cancer,","arzoxifene, Placebo,","Change in Masood Score, Number of Participants...",cancer,"Age, Continuous, Sex: Female, Male, Region of ...","`Age, Continuous`, `Sex: Female, Male`, `Regio...","`Age`, `Sex`, `Menopausal status`, `Hemoglobin...","`Age, Continuous`, `Sex: Female`, `Race/Ethnic...","`Age`, `Sex`, `Menopausal status`, `Body Mass ...","`Age, Continuous`, `Sex: Female`, `Race/Ethnic..."


In [4]:
length_df = pd.DataFrame()
for index, row in data_gen.iterrows():
    
    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    length_df.at[index, 'NCTId'] = row['NCTId']
    length_df.at[index, 'ref_length'] = len(module.extract_elements_v2(row['API_BaselineMeasures_Corrected']))
    length_df.at[index, 'gzs_length'] = len(module.extract_elements_v2(row['gpt4o_zs_gen']))
    length_df.at[index, 'gts_length'] = len(module.extract_elements_v2(row['gpt4o_ts_gen']))
    length_df.at[index, 'lzs_length'] = len(module.extract_elements_v2(row['llama3_70b_it_zs_gen']))
    length_df.at[index, 'lts_length'] = len(module.extract_elements_v2(row['llama3_70b_it_ts_gen']))
    

In [5]:
length_df.head(3)

,NCTId,ref_length,gzs_length,gts_length,lzs_length,lts_length
1,NCT00003901,11.0,18.0,12.0,11.0,11.0
2,NCT00005879,12.0,32.0,21.0,20.0,23.0
3,NCT00005908,6.0,16.0,14.0,17.0,14.0


In [6]:
data_gpt_eval.head(3)

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00003901,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C..."
2,NCT00005879,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C..."


In [7]:
import json
import pandas as pd

# Initialize an empty DataFrame with the specific column dtype as 'object'
gpt_match_len_df = pd.DataFrame(columns=['NCTId', 'gzs_match', 'gts_match', 'lzs_match', 'lts_match'], dtype='object')

for index, row in data_gpt_eval.iterrows():

    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    gpt_match_len_df.at[index, 'NCTId'] = row['NCTId']

    gzs = json.loads(row['gpt4o_zs_gen_matches'])
    # Assigning the tuple to the 'gzs_match' column
    gpt_match_len_df.at[index, 'gzs_match'] = (len(gzs['matched_features']), 
                                               len(gzs['remaining_reference_features']), 
                                               len(gzs['remaining_candidate_features']))
    
    gts = json.loads(row['gpt4o_ts_gen_matches'])
    # Assigning the tuple to the 'gts_match' column
    gpt_match_len_df.at[index, 'gts_match'] = (len(gts['matched_features']), 
                                               len(gts['remaining_reference_features']), 
                                               len(gts['remaining_candidate_features']))
    
    lzs = json.loads(row['llama3_70b_it_zs_gen_matches'])
    # Assigning the tuple to the 'lzs_match' column
    gpt_match_len_df.at[index, 'lzs_match'] = (len(lzs['matched_features']), 
                                               len(lzs['remaining_reference_features']), 
                                               len(lzs['remaining_candidate_features']))
    
    lts = json.loads(row['llama3_70b_it_ts_gen_matches'])
    # Assigning the tuple to the 'lts_match' column
    gpt_match_len_df.at[index, 'lts_match'] = (len(lts['matched_features']), 
                                               len(lts['remaining_reference_features']), 
                                               len(lts['remaining_candidate_features']))

print(gpt_match_len_df)


            NCTId   gzs_match   gts_match   lzs_match   lts_match
1     NCT00003901  (4, 7, 14)   (7, 4, 5)   (5, 6, 6)   (7, 4, 4)
2     NCT00005879  (4, 8, 28)  (6, 6, 15)  (5, 7, 15)  (6, 6, 17)
3     NCT00005908  (2, 4, 14)  (4, 2, 10)  (4, 2, 13)  (4, 2, 10)
4     NCT00006110  (3, 4, 16)  (5, 2, 10)  (4, 3, 12)  (4, 3, 11)
5     NCT00006392  (2, 6, 14)   (5, 3, 7)  (4, 4, 12)   (5, 3, 6)
...           ...         ...         ...         ...         ...
1688  NCT05204134  (3, 4, 12)   (4, 3, 6)  (2, 5, 12)   (3, 4, 8)
1689  NCT05289869   (4, 6, 7)   (6, 4, 5)   (6, 4, 9)   (4, 5, 6)
1690  NCT05387889  (4, 4, 11)   (4, 4, 7)  (3, 5, 12)   (4, 4, 6)
1691  NCT05451329   (2, 4, 8)   (3, 3, 5)   (4, 2, 6)   (3, 3, 5)
1692  NCT05741489  (3, 7, 13)   (4, 6, 9)   (5, 5, 8)   (4, 6, 5)

[1690 rows x 5 columns]


In [9]:
data_bert_eval.head(3)

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00126737,"{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""BMI""...","{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""Age""..."
2,NCT00283686,"{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""tota...","{\n ""matched_features"": [\n [\n ""tota...","{\n ""matched_features"": [\n [\n ""Age""..."


In [10]:
import json
import pandas as pd

# Initialize an empty DataFrame with the specific column dtype as 'object'
bert_match_len_df = pd.DataFrame(columns=['NCTId', 'gzs_match', 'gts_match', 'lzs_match', 'lts_match'], dtype='object')

for index, row in data_bert_eval.iterrows():

    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    bert_match_len_df.at[index, 'NCTId'] = row['NCTId']

    gzs = json.loads(row['gpt4o_zs_gen_matches'])
    # Assigning the tuple to the 'gzs_match' column
    bert_match_len_df.at[index, 'gzs_match'] = (len(gzs['matched_features']), 
                                               len(gzs['remaining_reference_features']), 
                                               len(gzs['remaining_candidate_features']))
    
    gts = json.loads(row['gpt4o_ts_gen_matches'])
    # Assigning the tuple to the 'gts_match' column
    bert_match_len_df.at[index, 'gts_match'] = (len(gts['matched_features']), 
                                               len(gts['remaining_reference_features']), 
                                               len(gts['remaining_candidate_features']))
    
    lzs = json.loads(row['llama3_70b_it_zs_gen_matches'])
    # Assigning the tuple to the 'lzs_match' column
    bert_match_len_df.at[index, 'lzs_match'] = (len(lzs['matched_features']), 
                                               len(lzs['remaining_reference_features']), 
                                               len(lzs['remaining_candidate_features']))
    
    lts = json.loads(row['llama3_70b_it_ts_gen_matches'])
    # Assigning the tuple to the 'lts_match' column
    bert_match_len_df.at[index, 'lts_match'] = (len(lts['matched_features']), 
                                               len(lts['remaining_reference_features']), 
                                               len(lts['remaining_candidate_features']))

print(bert_match_len_df)


           NCTId    gzs_match    gts_match    lzs_match    lts_match
1    NCT00126737    (3, 9, 6)  (2, 10, 18)   (2, 10, 9)    (3, 9, 9)
2    NCT00283686  (2, 36, 19)  (5, 33, 14)  (5, 33, 11)  (4, 34, 16)
3    NCT00329030   (3, 5, 20)   (4, 4, 12)   (4, 4, 16)   (5, 3, 11)
4    NCT00360334    (6, 3, 9)   (7, 2, 12)    (6, 3, 9)   (6, 3, 14)
5    NCT00395746   (5, 0, 10)   (5, 0, 13)    (4, 1, 9)    (5, 0, 8)
..           ...          ...          ...          ...          ...
97   NCT03708770   (2, 8, 23)   (4, 6, 12)   (4, 6, 15)   (4, 6, 10)
98   NCT03890588   (4, 6, 15)   (6, 4, 12)   (5, 5, 12)    (7, 3, 9)
99   NCT03987919    (9, 5, 9)   (7, 7, 10)    (7, 7, 7)   (9, 5, 11)
101  NCT04392375  (5, 22, 11)  (5, 22, 12)  (7, 20, 11)  (4, 23, 12)
102  NCT04823949   (4, 7, 16)   (5, 6, 12)   (4, 7, 14)   (5, 6, 11)

[100 rows x 5 columns]


## Unit Test Block
https://docs.python.org/3/library/unittest.html

In [8]:
import unittest

class TestMatches(unittest.TestCase):
    
    #NOTE: gen row comes from length_df, eval row comes from gpt_match_len_df
    def test_gzs_evals(self, gen_row, eval_row):
        self.assertEqual(gen_row['ref_length'], eval_row['gzs_match'][0] + eval_row['gzs_match'][1], 
                         f'GZS Reference length mismatch for {gen_row["NCTId"]}')
        self.assertEqual(gen_row['gzs_length'], eval_row['gzs_match'][0] + eval_row['gzs_match'][2],
                         f'GZS Candidate length mismatch for {gen_row["NCTId"]}')
        
    def test_gts_evals(self, gen_row, eval_row):   
        self.assertEqual(gen_row['ref_length'], eval_row['gts_match'][0] + eval_row['gts_match'][1], 
                         f'GTS Reference length mismatch for {gen_row["NCTId"]}')
        self.assertEqual(gen_row['gts_length'], eval_row['gts_match'][0] + eval_row['gts_match'][2],
                         f'GTS Candidate length mismatch for {gen_row["NCTId"]}')
        
    def test_lzs_evals(self, gen_row, eval_row):
        self.assertEqual(gen_row['ref_length'], eval_row['lzs_match'][0] + eval_row['lzs_match'][1], 
                         f'LZS Reference length mismatch for {gen_row["NCTId"]}')
        self.assertEqual(gen_row['lzs_length'], eval_row['lzs_match'][0] + eval_row['lzs_match'][2],
                         f'LZS Candidate length mismatch for {gen_row["NCTId"]}')
        
    def test_lts_evals(self, gen_row, eval_row):
        self.assertEqual(gen_row['ref_length'], eval_row['lts_match'][0] + eval_row['lts_match'][1],
                            f'LTS Reference length mismatch for {gen_row["NCTId"]}')
        self.assertEqual(gen_row['lts_length'], eval_row['lts_match'][0] + eval_row['lts_match'][2],
                            f'LTS Candidate length mismatch for {gen_row["NCTId"]}')



## Run Unit Test for GPT4 Evaluation

In [9]:
# Initialize the tester and the list to track IDs with exceptions
tester = TestMatches()
gpt4_exception_ids = []

count = 0

for v1, v2 in zip(length_df.iterrows(), gpt_match_len_df.iterrows()):

    #gzs test block
    try:
        tester.test_gzs_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"{count}. Assertion failed: {e}")
        count +=1
        # Append the NCTId of the row causing the exception to the list
        gpt4_exception_ids.append([v1[1]["NCTId"], 'gzs'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"{count}. An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        count +=1
        gpt4_exception_ids.append([v1[1]["NCTId"], 'gzs'])

    #gzs test block
    try:
        tester.test_gts_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"{count}. Assertion failed: {e}")
        count +=1
        # Append the NCTId of the row causing the exception to the list
        gpt4_exception_ids.append([v1[1]["NCTId"], 'gts'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"{count}. An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        count +=1
        gpt4_exception_ids.append([v1[1]["NCTId"], 'gts'])

    #lzs test block
    try:
        tester.test_lzs_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"{count}. Assertion failed: {e}")
        count +=1
        # Append the NCTId of the row causing the exception to the list
        gpt4_exception_ids.append([v1[1]["NCTId"], 'lzs'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"{count}. An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        count +=1
        gpt4_exception_ids.append([v1[1]["NCTId"], 'lzs'])

    #lts test block
    try:
        tester.test_lts_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"{count}. Assertion failed: {e}")
        count +=1
        # Append the NCTId of the row causing the exception to the list
        gpt4_exception_ids.append([v1[1]["NCTId"], 'lts'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"{count}. An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        count +=1
        gpt4_exception_ids.append([v1[1]["NCTId"], 'lts'])

# After the loop, you can print or further process the exception_ids
print("IDs that caused exceptions:", gpt4_exception_ids)

0. Assertion failed: 9.0 != 15 : GZS Reference length mismatch for NCT00079274
1. Assertion failed: 9.0 != 11 : GZS Reference length mismatch for NCT00082368
2. Assertion failed: 6.0 != 5 : GTS Reference length mismatch for NCT00088530
3. Assertion failed: 30.0 != 32 : GZS Candidate length mismatch for NCT00104572
4. Assertion failed: 14.0 != 15 : GTS Candidate length mismatch for NCT00104572
5. Assertion failed: 24.0 != 26 : LZS Candidate length mismatch for NCT00104572
6. Assertion failed: 14.0 != 15 : LTS Candidate length mismatch for NCT00109031
7. Assertion failed: 30.0 != 31 : GZS Candidate length mismatch for NCT00110084
8. Assertion failed: 20.0 != 21 : GZS Candidate length mismatch for NCT00116779
9. Assertion failed: 8.0 != 11 : GZS Reference length mismatch for NCT00116831
10. Assertion failed: 17.0 != 18 : GZS Candidate length mismatch for NCT00125853
11. Assertion failed: 14.0 != 15 : GTS Candidate length mismatch for NCT00125853
12. Assertion failed: 18.0 != 19 : LZS Cand

In [10]:
model_error = {'gzs':0, 'gts':0, 'lzs':0, 'lts':0}

for errors in gpt4_exception_ids:
    idn, model = errors[0], errors[1]
    model_error[model] += 1

print(model_error)
    

{'gzs': 145, 'gts': 125, 'lzs': 177, 'lts': 108}


## Run Unit Test for BERT Evaluation

In [14]:
# Initialize the tester and the list to track IDs with exceptions
tester = TestMatches()
bert_exception_ids = []

for v1, v2 in zip(length_df.iterrows(), bert_match_len_df.iterrows()):

    #gzs test block
    try:
        tester.test_gzs_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"Assertion failed: {e}")
        # Append the NCTId of the row causing the exception to the list
        bert_exception_ids.append([v1[1]["NCTId"], 'gzs'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        bert_exception_ids.append([v1[1]["NCTId"], 'gzs'])

    #gts test block
    try:
        tester.test_gts_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"Assertion failed: {e}")
        # Append the NCTId of the row causing the exception to the list
        bert_exception_ids.append([v1[1]["NCTId"], 'gts'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        bert_exception_ids.append([v1[1]["NCTId"], 'gts'])

    #lzs test block
    try:
        tester.test_lzs_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"Assertion failed: {e}")
        # Append the NCTId of the row causing the exception to the list
        bert_exception_ids.append([v1[1]["NCTId"], 'lzs'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        bert_exception_ids.append([v1[1]["NCTId"], 'lzs'])

    #lts test block
    try:
        tester.test_lts_evals(v1[1], v2[1])
    except AssertionError as e:
        # Print the exception message
        print(f"Assertion failed: {e}")
        # Append the NCTId of the row causing the exception to the list
        bert_exception_ids.append([v1[1]["NCTId"], 'lts'])
    except Exception as e:
        # Catch any other exceptions, print the message, and save the NCTId
        print(f"An error occurred for NCTId {v1[1]['NCTId']}: {e}")
        bert_exception_ids.append([v1[1]["NCTId"], 'lts'])

# After the loop, you can print or further process the exception_ids
print("IDs that caused exceptions:", bert_exception_ids)

IDs that caused exceptions: []
